# Install packages

In [ ]:
from google.colab import drive # import drive from google colab
drive.mount("/content/drive") 
!pip install neo
!pip install matplotlib_scalebar
!pip install ffmpeg
!pip install Ipython --upgrade
!pip3 install setuptools==65.5.0
%load_ext autoreload
%autoreload 2

# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os, sys
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

    
import sys
 

from data_wrangling import specific_utils
from null_behaviors import sample_null_distributions


import os, sys
import warnings
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from math import pi
from matplotlib import rc
from numpy import random

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

# Null distributions of features

Then I can compare the null distribution to the monkey's distribution

## Median time

**Median time**: we can find that an ideal observer can get to fireflies at this speed. Then we can find the median time of the ideal observer, and express the median time of the monkey as a fraction of the median time of the ideal observer

**Assumptions**: assume fireflies are fully visible (or the ideal observer has perfect memory): calculate the median time of going from one firefly to the nearest firefly (or firefly that takes least amount of time). Also assume quadratic action cost; so constant velocity



In [ ]:
all_median_time = returned_dist[0]

In [ ]:
median = agg_feature_statistics.loc[agg_feature_statistics['Item']=='t', 'Median'].item()

### run function

In [ ]:
all_median_time, all_median_distance, all_median_abs_angle, all_median_abs_angle_boundary, \
all_mean_time, all_mean_distance, all_mean_abs_angle, all_mean_abs_angle_boundary, all_total_time = sample_null_distributions.sample_null_distributions_func()


In [ ]:
print(np.array(agg_feature_statistics['Label for median']))

In [ ]:
all_null_distr = {'all_median_time': all_median_time,
                 'all_median_distance': all_median_distance,
                 'all_median_abs_angle': all_median_abs_angle, 
                 'all_median_abs_angle_boundary': all_median_abs_angle_boundary,
                 'all_mean_time': all_mean_time,
                 'all_mean_distance': all_mean_distance,
                 'all_mean_abs_angle':all_mean_abs_angle,
                 'all_mean_abs_angle_boundary': all_mean_abs_angle_boundary,
                 'all_total_time':all_total_time
}

Adding a line for monkey's statistic...so ugly...I can just print

In [ ]:
agg_feature_statistics.Item

In [ ]:

median = agg_feature_statistics.loc[agg_feature_statistics['Item']==item_name, 'Median'].item()
sns.histplot(null_distr, stat='percent')
#plt.plot([median, median], [0, 1], linewidth=2)

plt.show()

plt.title('Distribution of Median Distance from 1000 Samples')


In [ ]:
def plot_null_distr_median(null_distr = all_median_distance, item_name = 'd_last_vis'):

### Plot

In [ ]:
all_median_abs_angle

In [ ]:
null_distr = all_median_abs_angle
item_name = 'abs_angle_last_vis'
median = agg_feature_statistics.loc[agg_feature_statistics['Item']==item_name, 'Median'].item()
sns.histplot(null_distr, stat='percent')

plt.title('Distribution of Median Absolute Angle from 1000 Samples')
plt.show()
print('Monkey:', median)

In [ ]:
null_distr = all_median_distance
item_name = 'd_last_vis'
median = agg_feature_statistics.loc[agg_feature_statistics['Item']==item_name, 'Median'].item()
sns.histplot(null_distr, stat='percent')
plt.title('Distribution of Median Distance from 1000 Samples')
plt.show()

print('Monkey:', median)

In [ ]:
median = agg_feature_statistics.loc[agg_feature_statistics['Item']=='t', 'Median'].item()
sns.histplot(all_median_time, stat='percent')
#plt.plot([median, median], [0, 1], linewidth=2)
plt.title('Distribution of Median Time from 1000 Samples')
plt.show()

print('Monkey:', median)



sns.histplot(all_median_time, stat='percent')
#plt.plot([median, median], [0, 1], linewidth=2)
plt.title('Distribution of Median Time from 1000 Samples')
plt.show()

print('Monkey:', median)


sns.histplot(all_median_time, stat='percent')
#plt.plot([median, median], [0, 1], linewidth=2)
plt.title('Distribution of Median Time from 1000 Samples')
plt.show()

print('Monkey:', median)

sns.histplot(all_median_time, stat='percent')
#plt.plot([median, median], [0, 1], linewidth=2)
plt.title('Distribution of Median Time from 1000 Samples')
plt.show()

print('Monkey:', median)



I can probably save all_median_time since it takes a while

## ff_capture_rate

### All pooled

In [ ]:
len(all_min_time)/all_min_time.sum()

### By trials

In [ ]:
num_trials_per_sample/all_total_time

## Median abs angle

### All pooled

In [ ]:
# Sample 10000 fireflies that are in front of the monkey (have a positive y coordinate)
num_alive_ff = 10000
fftheta = random.rand(num_alive_ff) * pi
# Find the median of the absolute angles
median_abs_angle = np.median(np.abs(fftheta))
# And then find the medians of the absoluate angles to boundary
arena_radius = 1000
ffr = np.sqrt(random.rand(num_alive_ff)) * arena_radius
ffx = np.cos(fftheta) * ffr
ffy = np.sin(fftheta) * ffr
ffxy = np.stack((ffx, ffy), axis=1)
distances_to_ff = np.linalg.norm(ffxy, axis = 1)
angles_to_boundaries = specific_utils.calculate_angles_to_ff_boundaries(fftheta, distances_to_ff)
median_abs_angle_to_boundary = np.median(np.abs(angles_to_boundaries))
print("median_abs_angle is", median_abs_angle)
print("median_abs_angle_to_boundary is", median_abs_angle_to_boundary)
sns.histplot(fftheta)
plt.show() 
sns.histplot(angles_to_boundaries)
plt.show() 

### by trials

In [ ]:
# Sample 10000 fireflies that are in front of the monkey (have a positive y coordinate)
num_alive_ff = 10000 
fftheta = random.rand(num_alive_ff) * pi
# Find the median of the absolute angles
median_abs_angle = np.median(np.abs(fftheta))
# And then find the medians of the absoluate angles to boundary
arena_radius = 1000
ffr = np.sqrt(random.rand(num_alive_ff)) * arena_radius
ffx = np.cos(fftheta) * ffr
ffy = np.sin(fftheta) * ffr
ffxy = np.stack((ffx, ffy), axis=1)
distances_to_ff = np.linalg.norm(ffxy, axis = 1)
angles_to_boundaries = specific_utils.calculate_angles_to_ff_boundaries(fftheta, distances_to_ff)
median_abs_angle_to_boundary = np.median(np.abs(angles_to_boundaries))
print("median_abs_angle is", median_abs_angle)
print("median_abs_angle_to_boundary is", median_abs_angle_to_boundary)
sns.histplot(fftheta)
plt.show()
sns.histplot(angles_to_boundaries)
plt.show()

## Median distance

In [ ]:
num_alive_ff = 10000
arena_radius = 1000
ffr = np.sqrt(random.rand(num_alive_ff)) * arena_radius
sns.histplot(ffr)
plt.show()
np.median(ffr)

(['two_in_a_row', 'visible_before_last_one', 'disappear_latest', 'waste_cluster_around_target',
                                                                   'ignore_sudden_flash', 'give_up_after_trying', 'try_a_few_times', 'ff_capture_rate', 'stop_success_rate'])]

## two_in_a_row

Suppose the agent can pick up all fireflies in a row. So I guess this question comes to the percentage of clusters in the arena 

I can use algorithm from cluster_analysis.make_point_vs_cluster 

In [ ]:
scenario_rates = []
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    max_cluster_distance = 50
    # For each of 10000 samples 
    all_min_time = []
    for sample in range(100):
        # Suppose the monkey is at the orgin, sample 200 fireflies following poisson distribution
        num_alive_ff = 200
        arena_radius = 1000
        fftheta = random.rand(num_alive_ff) * 2 * pi
        # And then sample a radius for each ff and get the x, y coordinates
        ffr = np.sqrt(random.rand(len(fftheta))) * arena_radius
        ffx = np.cos(fftheta) * ffr
        ffy = np.sin(fftheta) * ffr
        ffxy = np.stack((ffx, ffy), axis=1)


        # Use the function scipy.cluster.hierarchy.linkage
        # method = "single" means that we're using the Nearest Point Algorithm
        # In the returned array, each row is a procedure of combining two components into one cluster (the closer two components are, the sooner they'll be combined); 
        # there are n-1 rows in total in the returned variable
        linked = linkage(ffxy, method='single')

        # In order to find the number of clusters that satisfy our requirements, we'll first find the number of rows from linked
        # where the two components of the cluster are greater than max_cluster_distance (100 cm as default). 
        # If this number is n, then it means that at one point in the procedure, there are n+1 clusters that all satisfy our requirements, 
        # and then these clusters are condensed into 1 big cluster through combining two clusters into one and repearting the procedure;
        # However, these final steps of combining are invalid for our use since all the n+1 clusters are further apart than max_cluster_distance;
        # Thus, we know that we can separate the fireflies into at most n+1 clusters that satisfy our requirement
        # that each firefly is within 50 cm of its nearest firefly in the cluster.
        num_clusters = sum(linked[:, 2] > max_cluster_distance)+1  


        # This time, we assign the fireflies into the number of clusters we just found, so that we know which cluster each firefly belongs to.
        # The variable "cluster_labels" contains the cluster label for each firefly
        cluster_labels = AgglomerativeClustering(n_clusters=num_clusters, affinity='euclidean', linkage='single').fit_predict(ffxy)
        # Find the clusters that have more than one firefly
        unique_cluster_labels, ff_counts = np.unique(cluster_labels, return_counts=True)
        clusters_w_more_than_1_ff = ff_counts[ff_counts > 1]
        two_in_a_row_scenario_rate = (np.sum(clusters_w_more_than_1_ff) - len(clusters_w_more_than_1_ff))/200
        scenario_rates.append(two_in_a_row_scenario_rate)

scenario_rates = np.array(scenario_rates)
sns.histplot(scenario_rates)
plt.show()
np.median(scenario_rates)